In [2]:
import torch
# GPU 사용 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
print(torch.cuda.get_device_name(0))

# CUDA 사용 가능 여부 확인
print(f"CUDA available: {torch.cuda.is_available()}")  # True여야 함

# PyTorch에서 사용하는 CUDA 버전 확인
print(f"CUDA version in PyTorch: {torch.version.cuda}")

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

cuda
NVIDIA GeForce RTX 4050 Laptop GPU
CUDA available: True
CUDA version in PyTorch: 11.8


In [3]:
# 라이브러리 임포트
import numpy as np
import pandas as pd
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import albumentations as A
import matplotlib.pyplot as plt
import torch.nn.functional as F

d:\isy\test\Lib\site-packages\albumentations\__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.18 (you have 1.4.17). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [38]:
import torch
import cv2
import numpy as np
import os
import random
from torchvision import models, transforms

# ResNet 및 Inception 모델 수정
class CustomModel(torch.nn.Module):
    def __init__(self, model_name):
        super(CustomModel, self).__init__()
        if model_name == 'efficientnet':
            self.model = models.efficientnet_b1(weights='IMAGENET1K_V1') 
            self.model.classifier[1] = torch.nn.Linear(self.model.classifier[1].in_features, 1)  # 이진 분류
        
    def forward(self, x):
        return self.model(x)

# efficientnet 모델 로드
efficientnet_model = CustomModel(model_name='efficientnet')
efficientnet_model.eval()  # 평가 모드로 설정

# 전처리 파이프라인 정의
preprocess = transforms.Compose([
    transforms.ToPILImage(),  # OpenCV 이미지를 PIL 이미지로 변환
    transforms.Resize((240, 240)),  # 모델 입력 크기에 맞게 크기 조정
    transforms.ToTensor(),  # 텐서로 변환
])

# 이미지 전처리 함수
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # RGB로 변환
    image = preprocess(image)  # 전처리 적용
    image = image.unsqueeze(0)  # 배치 차원 추가
    return image

# 예측 함수 정의
def predict_image(model, image_path, device):
    model.to(DEVICE)  # 모델을 디바이스로 이동
    image = preprocess_image(image_path).to(device)  # 전처리 후 디바이스로 이동
    with torch.no_grad():  # 평가 모드에서는 gradient 계산 비활성화
        output = model(image)
        predicted_prob = torch.sigmoid(output).item()  # 확률로 변환

    # 0.5를 기준으로 정상/비정상 예측
    prediction = 'NG (결함 있음)' if predicted_prob > 0.5 else 'OK (정상)'
    return prediction, predicted_prob

# NG 폴더 경로
ng_folder_path = '../PCB_imgs/all/resize/NG/' 
# OK 폴더 경로
ok_folder_path = '../PCB_imgs/all/resize/OK/' 
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # 디바이스 설정

# 각 폴더에서 무작위로 10개의 이미지를 가져오기
def load_random_images(folder_path, num_images=10):
    images = os.listdir(folder_path)
    random_images = random.sample(images, min(num_images, len(images)))
    return [os.path.join(folder_path, img) for img in random_images]

# NG 및 OK 이미지 로드
ng_images = load_random_images(ng_folder_path)
ok_images = load_random_images(ok_folder_path)

# NG 이미지 예측 결과와 확률
print(f'NG 예측결과:')
for image_path in ng_images:
    prediction, prob = predict_image(efficientnet_model, image_path, DEVICE) 
    print(f"Efficientnet: {prediction}, 확률: {prob:.4f}")

# OK 이미지 예측 결과와 확률
print(f'OK 예측결과:')
for image_path in ok_images:
    prediction, prob = predict_image(efficientnet_model, image_path, DEVICE) 
    print(f"Efficientnet: {prediction}, 확률: {prob:.4f}")

NG 예측결과:
Efficientnet: NG (결함 있음), 확률: 0.6173
Efficientnet: OK (정상), 확률: 0.4670
Efficientnet: NG (결함 있음), 확률: 0.6463
Efficientnet: NG (결함 있음), 확률: 0.6715
Efficientnet: NG (결함 있음), 확률: 0.5973
Efficientnet: NG (결함 있음), 확률: 0.6222
Efficientnet: NG (결함 있음), 확률: 0.6606
Efficientnet: NG (결함 있음), 확률: 0.5893
Efficientnet: NG (결함 있음), 확률: 0.5827
Efficientnet: NG (결함 있음), 확률: 0.6079
OK 예측결과:
Efficientnet: NG (결함 있음), 확률: 0.5900
Efficientnet: NG (결함 있음), 확률: 0.5702
Efficientnet: OK (정상), 확률: 0.4841
Efficientnet: NG (결함 있음), 확률: 0.5781
Efficientnet: NG (결함 있음), 확률: 0.5663
Efficientnet: NG (결함 있음), 확률: 0.6192
Efficientnet: NG (결함 있음), 확률: 0.6229
Efficientnet: NG (결함 있음), 확률: 0.5889
Efficientnet: NG (결함 있음), 확률: 0.6276
Efficientnet: NG (결함 있음), 확률: 0.5725


In [4]:
from torchvision import datasets
import pandas as pd

IMAGE_SIZE = 240
BATCH_SIZE = 32
EPOCHS = 10

# 이미지 변환 정의
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),  # 이미지를 Tensor로 변환 (0~255 범위를 0~1 범위로 정규화)
])

# 데이터셋 디렉토리 설정
train_dir = '../PCB_imgs/all/resize/train'
val_dir = '../PCB_imgs/all/resize/validation'
test_dir = '../PCB_imgs/all/resize/test'

# ImageFolder로 데이터셋 불러오기
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
val_dataset = datasets.ImageFolder(root=val_dir, transform=transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)

# 파일 경로 및 타겟 추출
train_file_paths = [img[0] for img in train_dataset.imgs]
train_targets = train_dataset.targets

val_file_paths = [img[0] for img in val_dataset.imgs]
val_targets = val_dataset.targets

test_file_paths = [img[0] for img in test_dataset.imgs]
test_targets = test_dataset.targets

# DataFrame 생성
train_df = pd.DataFrame({'file_paths': train_file_paths, 'targets': train_targets})
validation_df = pd.DataFrame({'file_paths': val_file_paths, 'targets': val_targets})
test_df = pd.DataFrame({'file_paths': test_file_paths, 'targets': test_targets})

# 확인을 위해 각 데이터셋의 크기 출력
print(f"Train 데이터 수: {len(train_df)}")
print(f"Validation 데이터 수: {len(validation_df)}")
print(f"Test 데이터 수: {len(test_df)}")

Train 데이터 수: 2008
Validation 데이터 수: 502
Test 데이터 수: 628


In [5]:
import torch
from torchvision import datasets, transforms

# resize된 데이터셋 로드
input_dir = '../PCB_imgs/all/resize/'
dataset = datasets.ImageFolder(root=input_dir, transform=transform)

# 데이터 로더 정의
dataloader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

# 데이터 로더에서 데이터 가져오기
for images, labels in dataloader:
    print(images.shape)  # 배치의 이미지 텐서 크기 확인
    break  # 첫 번째 배치만 확인
    # 배치 크기, 채널 수(RGB), 이미지 크기 

torch.Size([32, 3, 240, 240])


In [6]:
# 커스텀 데이터세트 정의
class CustomDataset(Dataset):
    def __init__(self, file_paths, targets, aug=None, preprocess=None):
        self.file_paths = file_paths
        self.targets = targets
        self.aug = aug
        self.preprocess = preprocess

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, index):
        file_path = self.file_paths[index]
        target = self.targets[index]
        
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        image = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE))

        if self.aug is not None:
            image = self.aug(image=image)['image']

        if self.preprocess is not None:
            image = self.preprocess(image)

        image = np.transpose(image, (2, 0, 1))  # (H, W, C) -> (C, H, W)
        image = torch.tensor(image, dtype=torch.float32)
        
        return image, target

In [7]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import DataLoader

# 데이터셋 인스턴스 생성
train_dataset = CustomDataset(train_df['file_paths'].values, train_df['targets'].values)
validation_dataset = CustomDataset(validation_df['file_paths'].values, validation_df['targets'].values)
test_dataset = CustomDataset(test_df['file_paths'].values, test_df['targets'].values)

# DataLoader 설정
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [7]:
# 평가 함수 정의
def evaluate_model(model, test_loader, criterion):
    model.eval()
    running_test_loss = 0.0
    correct_test = 0
    total_test = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device).float()
            outputs = model(images)
            outputs = outputs.view(-1) # torch.Size([batch_size])로 변환
            loss = criterion(outputs, labels)  # 손실 계산
            running_test_loss += loss.item()
            
            predicted = (torch.sigmoid(outputs) > 0.5).float()  # 0.5 기준으로 이진 분류
            total_test += labels.size(0)
            correct_test += (predicted == labels).sum().item()
            
    # 검증 손실과 정확도
    test_loss = running_test_loss / len(test_loader)
    test_accuracy = correct_test / total_test
    return test_loss, test_accuracy

efficientnet b1 구조만 이용

In [9]:
import torch
import torch.nn as nn
import torchvision

# 모델 정의
class CustomModel(nn.Module):
    def __init__(self, model_name='efficientnet'):
        super(CustomModel, self).__init__()
        if model_name == 'efficientnet':
            self.base_model = torchvision.models.efficientnet_b1()
            self.base_model.classifier = nn.Identity()  # 마지막 분류기 제거
        elif model_name == 'resnet50':
            self.base_model = torchvision.models.resnet50()
            self.base_model = nn.Sequential(*list(self.base_model.children())[:-1])  # 마지막 레이어 제거
        elif model_name == 'inception':
            self.base_model = torchvision.models.inception_v3()
            self.base_model.classifier = nn.Identity()  # 마지막 분류기 제거

        self.fc1 = nn.Linear(self._get_features_dim(model_name), 1)

    def _get_features_dim(self, model_name):
        if model_name == 'efficientnet':
            return 1280  # EfficientNet의 출력 차원
        elif model_name == 'resnet50':
            return 2048  # ResNet50의 출력 차원
        elif model_name == 'inception':
            return 2048  # Inception의 출력 차원

    def forward(self, x):
        x = self.base_model(x)
        
        if isinstance(x, tuple): 
            x = x[0]  

        x = x.view(x.size(0), -1)  
        x = self.fc1(x)  

        return x 

In [10]:
# 모델 초기화
model = CustomModel(model_name='efficientnet').to(DEVICE)
# 손실 함수 및 최적화함수 정의
criterion = nn.BCEWithLogitsLoss()  # 이진 교차 엔트로피 손실
optimizer = optim.Adam(model.parameters(), lr=0.0001)  # Adam 옵티마이저

In [11]:
EPOCHS = 20

# 조기 종료 변수를 초기화합니다.
best_val_loss = float('inf')
patience = 5  # 개선이 없을 때 기다릴 에포크 수
patience_counter = 0

# 모델 훈련
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for images, targets in train_loader:
        images, targets = images.to(DEVICE), targets.to(DEVICE).float()
        
        optimizer.zero_grad()
        outputs = model(images)

        # 손실 계산
        loss = criterion(outputs.view(-1), targets)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

        # 정확도 계산
        predicted = (torch.sigmoid(outputs.view(-1)) > 0.5).float()
        correct_predictions += (predicted == targets).sum().item()
        total_predictions += targets.size(0)

    # 훈련 데이터 정확도
    train_accuracy = correct_predictions / total_predictions

    # 검증
    val_loss, val_accuracy = evaluate_model(model, validation_loader, criterion)

    print(f'Epoch [{epoch+1}/{EPOCHS}], Train Loss: {running_loss / len(train_loader):.4f}, '
          f'Train Accuracy: {train_accuracy:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

    # 조기 종료 로직
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0  # 손실 개선 시 카운터 리셋
        print("Validation loss improved, saving model...")
    else:
        patience_counter += 1  # 손실이 개선되지 않으면 카운터 증가

    if patience_counter >= patience:
        print("Early stopping triggered.")
        break  # 훈련 종료

Epoch [1/20], Train Loss: 0.6810, Train Accuracy: 0.6036, Val Loss: 0.6875, Val Accuracy: 0.5857
Validation loss improved, saving model...
Epoch [2/20], Train Loss: 0.5166, Train Accuracy: 0.7440, Val Loss: 0.3891, Val Accuracy: 0.8048
Validation loss improved, saving model...
Epoch [3/20], Train Loss: 0.4108, Train Accuracy: 0.8123, Val Loss: 0.3514, Val Accuracy: 0.8426
Validation loss improved, saving model...
Epoch [4/20], Train Loss: 0.3762, Train Accuracy: 0.8222, Val Loss: 0.3457, Val Accuracy: 0.8426
Validation loss improved, saving model...
Epoch [5/20], Train Loss: 0.3303, Train Accuracy: 0.8616, Val Loss: 0.3766, Val Accuracy: 0.8327
Epoch [6/20], Train Loss: 0.2628, Train Accuracy: 0.8889, Val Loss: 0.4501, Val Accuracy: 0.8327
Epoch [7/20], Train Loss: 0.2611, Train Accuracy: 0.8884, Val Loss: 0.3786, Val Accuracy: 0.8606
Epoch [8/20], Train Loss: 0.2180, Train Accuracy: 0.9064, Val Loss: 0.3962, Val Accuracy: 0.8506
Epoch [9/20], Train Loss: 0.1990, Train Accuracy: 0.9228

In [12]:
test_loss, test_accuracy = evaluate_model(model, test_loader, criterion)
print(f'Test loss: {test_loss:.4f}, Test accuracy: {test_accuracy:.4f}')

Test loss: 0.3406, Test accuracy: 0.8710


##### Efficientnet + Imagenet 파라미터

In [ ]:
import torch
import torch.nn as nn
import torchvision

# 모델 정의
class CustomModel(nn.Module):
    def __init__(self, model_name='efficientnet'):
        super(CustomModel, self).__init__()
        if model_name == 'efficientnet':
            self.base_model = torchvision.models.efficientnet_b1(weights='IMAGENET1K_V1')
            self.base_model.classifier = nn.Identity()  # 마지막 분류기 제거
        elif model_name == 'resnet50':
            self.base_model = torchvision.models.resnet50(weights='IMAGENET1K_V1')
            self.base_model = nn.Sequential(*list(self.base_model.children())[:-1])  # 마지막 레이어 제거
        elif model_name == 'inception':
            self.base_model = torchvision.models.inception_v3(weights='IMAGENET1K_V1')
            self.base_model.classifier = nn.Identity()  # 마지막 분류기 제거

        self.fc1 = nn.Linear(self._get_features_dim(model_name), 1)

    def _get_features_dim(self, model_name):
        if model_name == 'efficientnet':
            return 1280  # EfficientNet의 출력 차원
        elif model_name == 'resnet50':
            return 2048  # ResNet50의 출력 차원
        elif model_name == 'inception':
            return 2048  # Inception의 출력 차원

    def forward(self, x):
        x = self.base_model(x)
        if isinstance(x, tuple): 
            x = x[0] 
        
        x = x.view(x.size(0), -1)  
        x = self.fc1(x)  

        return x 

In [13]:
# 모델 초기화
model = CustomModel(model_name='efficientnet').to(DEVICE)
# 손실 함수 및 최적화함수 정의
criterion = nn.BCEWithLogitsLoss()  # 이진 교차 엔트로피 손실
optimizer = optim.Adam(model.parameters(), lr=0.0001)  # Adam 옵티마이저

In [14]:
EPOCHS = 20

# 조기 종료 변수를 초기화합니다.
best_val_loss = float('inf')
patience = 5  # 개선이 없을 때 기다릴 에포크 수
patience_counter = 0

# 모델 훈련
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for images, targets in train_loader:
        images, targets = images.to(DEVICE), targets.to(DEVICE).float()
        
        optimizer.zero_grad()
        outputs = model(images)

        # 손실 계산
        loss = criterion(outputs.view(-1), targets)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

        # 정확도 계산
        predicted = (torch.sigmoid(outputs.view(-1)) > 0.5).float()
        correct_predictions += (predicted == targets).sum().item()
        total_predictions += targets.size(0)

    # 훈련 데이터 정확도
    train_accuracy = correct_predictions / total_predictions

    # 검증
    val_loss, val_accuracy = evaluate_model(model, validation_loader, criterion)

    print(f'Epoch [{epoch+1}/{EPOCHS}], Train Loss: {running_loss / len(train_loader):.4f}, '
          f'Train Accuracy: {train_accuracy:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

    # 조기 종료 로직
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0  # 손실 개선 시 카운터 리셋
        print("Validation loss improved, saving model...")
    else:
        patience_counter += 1  # 손실이 개선되지 않으면 카운터 증가

    if patience_counter >= patience:
        print("Early stopping triggered.")
        break  # 훈련 종료

Epoch [1/20], Train Loss: 0.6522, Train Accuracy: 0.6419, Val Loss: 0.6797, Val Accuracy: 0.5857
Validation loss improved, saving model...
Epoch [2/20], Train Loss: 0.5045, Train Accuracy: 0.7605, Val Loss: 0.4254, Val Accuracy: 0.7988
Validation loss improved, saving model...
Epoch [3/20], Train Loss: 0.4316, Train Accuracy: 0.8038, Val Loss: 0.3448, Val Accuracy: 0.8566
Validation loss improved, saving model...
Epoch [4/20], Train Loss: 0.3708, Train Accuracy: 0.8327, Val Loss: 0.4352, Val Accuracy: 0.8347
Epoch [5/20], Train Loss: 0.3190, Train Accuracy: 0.8601, Val Loss: 0.3762, Val Accuracy: 0.8486
Epoch [6/20], Train Loss: 0.2832, Train Accuracy: 0.8820, Val Loss: 0.3614, Val Accuracy: 0.8426
Epoch [7/20], Train Loss: 0.2170, Train Accuracy: 0.9143, Val Loss: 0.4275, Val Accuracy: 0.8506
Epoch [8/20], Train Loss: 0.2001, Train Accuracy: 0.9208, Val Loss: 0.4819, Val Accuracy: 0.8546
Early stopping triggered.


In [15]:
# 테스트 데이터세트로 평가
test_loss, test_accuracy = evaluate_model(model, test_loader, criterion)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')

Test Loss: 0.3495, Test Accuracy: 0.8599


훈련데이터의 손실값은 계속 줄고, 정확도는 증가하는 반면, 검증 데이터의 손실값과 정확도는 줄지않음.  
##### 데이터 증강, dropout 추가

In [16]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import DataLoader

# 데이터 증강 정의
aug = A.Compose([
    A.HorizontalFlip(p=0.5),                # 좌우 반전
    A.VerticalFlip(p=0.5),                  # 상하 반전
    A.Rotate(limit=10, p=0.5),              # 작은 각도 회전 (10도 내외)
    A.RandomBrightnessContrast(p=0.5),      # 밝기 및 대비 조절
])

# 데이터셋 인스턴스 생성
train_dataset = CustomDataset(train_df['file_paths'].values, train_df['targets'].values, aug=aug)
validation_dataset = CustomDataset(validation_df['file_paths'].values, validation_df['targets'].values)
test_dataset = CustomDataset(test_df['file_paths'].values, test_df['targets'].values)

# DataLoader 설정
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [17]:
# 모델 정의
class CustomModel(nn.Module):
    def __init__(self, model_name='efficientnet'):
        super(CustomModel, self).__init__()
        if model_name == 'efficientnet':
            self.base_model = torchvision.models.efficientnet_b1(weights='IMAGENET1K_V1')
            self.base_model.classifier = nn.Identity()  # 마지막 분류기 제거
        elif model_name == 'resnet50':
            self.base_model = torchvision.models.resnet50(weights='IMAGENET1K_V1')
            self.base_model = nn.Sequential(*list(self.base_model.children())[:-1])  # 마지막 레이어 제거
        elif model_name == 'inception':
            self.base_model = torchvision.models.inception_v3(weights='IMAGENET1K_V1')
            self.base_model.classifier = nn.Identity()  # 마지막 분류기 제거

        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(self._get_features_dim(model_name), 1)

    def _get_features_dim(self, model_name):
        if model_name == 'efficientnet':
            return 1280  # EfficientNet B0의 출력 차원
        elif model_name == 'resnet50':
            return 2048  # ResNet50의 출력 차원
        elif model_name == 'inception':
            return 2048  # Inception의 출력 차원

    def forward(self, x):
        x = self.base_model(x)
        
        # Inception 모델에 대한 수정
        if isinstance(x, tuple):  # Inception 모델이 여러 출력을 반환하는 경우
            x = x[0]  # 첫 번째 출력을 선택
        
        x = x.view(x.size(0), -1)  # Flatten
        x = self.dropout(x)
        x = self.fc1(x)  # Sigmoid 제거

        return x  # 최종 출력

In [18]:
# 모델 초기화
model = CustomModel(model_name='efficientnet').to(DEVICE)
# 손실 함수 및 최적화함수 정의
criterion = nn.BCEWithLogitsLoss()  # 이진 교차 엔트로피 손실
optimizer = optim.Adam(model.parameters(), lr=0.0001)  # Adam 옵티마이저

In [19]:
EPOCHS = 20

# 조기 종료 변수를 초기화합니다.
best_val_loss = float('inf')
patience = 5  # 개선이 없을 때 기다릴 에포크 수
patience_counter = 0

# 모델 훈련
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for images, targets in train_loader:
        images, targets = images.to(DEVICE), targets.to(DEVICE).float()
        
        optimizer.zero_grad()
        outputs = model(images)

        # 손실 계산
        loss = criterion(outputs.view(-1), targets)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

        # 정확도 계산
        predicted = (torch.sigmoid(outputs.view(-1)) > 0.5).float()
        correct_predictions += (predicted == targets).sum().item()
        total_predictions += targets.size(0)

    # 훈련 데이터 정확도
    train_accuracy = correct_predictions / total_predictions

    # 검증
    val_loss, val_accuracy = evaluate_model(model, validation_loader, criterion)

    print(f'Epoch [{epoch+1}/{EPOCHS}], Train Loss: {running_loss / len(train_loader):.4f}, '
          f'Train Accuracy: {train_accuracy:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

    # 조기 종료 로직
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0  # 손실 개선 시 카운터 리셋
        print("Validation loss improved, saving model...")
        # 모델 저장
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_loss': val_loss,
        }, '../model/efficientnet_b1_best_model.pth')
    else:
        patience_counter += 1  # 손실이 개선되지 않으면 카운터 증가

    if patience_counter >= patience:
        print("Early stopping triggered.")
        break  # 훈련 종료

Epoch [1/20], Train Loss: 0.5063, Train Accuracy: 0.7560, Val Loss: 0.3639, Val Accuracy: 0.8486
Validation loss improved, saving model...
Epoch [2/20], Train Loss: 0.3296, Train Accuracy: 0.8471, Val Loss: 0.2887, Val Accuracy: 0.8924
Validation loss improved, saving model...
Epoch [3/20], Train Loss: 0.2515, Train Accuracy: 0.8949, Val Loss: 0.2420, Val Accuracy: 0.9084
Validation loss improved, saving model...
Epoch [4/20], Train Loss: 0.1999, Train Accuracy: 0.9228, Val Loss: 0.2100, Val Accuracy: 0.9323
Validation loss improved, saving model...
Epoch [5/20], Train Loss: 0.1448, Train Accuracy: 0.9422, Val Loss: 0.1781, Val Accuracy: 0.9402
Validation loss improved, saving model...
Epoch [6/20], Train Loss: 0.1107, Train Accuracy: 0.9626, Val Loss: 0.1527, Val Accuracy: 0.9422
Validation loss improved, saving model...
Epoch [7/20], Train Loss: 0.0964, Train Accuracy: 0.9681, Val Loss: 0.1551, Val Accuracy: 0.9402
Epoch [8/20], Train Loss: 0.0832, Train Accuracy: 0.9731, Val Loss: 0

In [20]:
# 모델 클래스 정의 (CustomModel)
model = CustomModel(model_name='efficientnet').to(DEVICE)

# 저장된 체크포인트 불러오기
checkpoint = torch.load('../model/efficientnet_b1_best_model.pth')

# 모델에 체크포인트 적용
model.load_state_dict(checkpoint['model_state_dict'])

optimizer = optim.Adam(model.parameters(), lr=0.0001)  # Adam 옵티마이저

# 모델을 평가 모드로 전환
model.eval()

C:\Users\enssel\AppData\Local\Temp\ipykernel_724\3435278120.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('../model/efficientnet_b1_best_model.

CustomModel(
  (base_model): EfficientNet(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): MBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
              (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
            (1): SqueezeExcitation(
              (avgpool): AdaptiveAvgPool2d(output_size=1)
              (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
              (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
              (activation): SiLU(inplace=True)
              (scal

In [21]:
# 테스트 데이터세트로 평가
test_loss, test_accuracy = evaluate_model(model, test_loader, criterion)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')

Test Loss: 0.0996, Test Accuracy: 0.9650


오버샘플링 + 증강 + bn층 추가 + 학습률 스케줄러 + l2 규제

In [32]:
import os
from PIL import Image 

class OverSamplingDataset(Dataset):
    def __init__(self, ng_dir, ok_dir, transform=None, aug=None):
        self.ng_images = [os.path.join(ng_dir, img) for img in os.listdir(ng_dir)]
        self.ok_images = [os.path.join(ok_dir, img) for img in os.listdir(ok_dir)]
        self.transform = transform
        self.aug = aug

        # NG 이미지 수와 OK 이미지 수 출력
        print(f'NG 이미지 수: {len(self.ng_images)}')
        print(f'OK 이미지 수: {len(self.ok_images)}')

        # OK 이미지를 NG 이미지 수 만큼 반복하여 오버샘플링
        self.ok_images = self.ok_images * (len(self.ng_images) // len(self.ok_images)) + self.ok_images[:len(self.ng_images) % len(self.ok_images)]

        # 오버샘플링 후 OK 이미지 수가 830이 되도록 맞춤
        self.ok_images = self.ok_images[:len(self.ng_images)]  # NG와 동일한 수로 제한

        # 최종 데이터셋은 NG와 OK 이미지의 합
        self.images = self.ng_images + self.ok_images
        self.labels = [0] * len(self.ng_images) + [1] * len(self.ok_images)  # NG=0, OK=1

        # 최종 이미지와 레이블 수 출력
        print(f'오버샘플링 후 NG 이미지 수: {len(self.ng_images)}')
        print(f'오버샘플링 후 OK 이미지 수: {len(self.ok_images)}')

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).convert("RGB")  # PIL 이미지로 열기

        if self.aug is not None:
            # PIL 이미지를 NumPy 배열로 변환
            image = np.array(image)  # np.array로 변환
            image = self.aug(image=image)['image']  # 데이터 증강 적용
            image = Image.fromarray(image)  # 다시 PIL 이미지로 변환

        if self.transform:
            image = self.transform(image)

        label = self.labels[idx]
        return image, label

# 데이터 변환 정의
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
])

# 데이터 증강 정의
aug = A.Compose([
    A.HorizontalFlip(p=0.6),                # 좌우 반전
    A.VerticalFlip(p=0.6),                  # 상하 반전
    A.Rotate(limit=10, p=0.6),              # 작은 각도 회전 (10도 내외)
    A.RandomBrightnessContrast(p=0.6),      # 밝기 및 대비 조절
])

# 데이터셋 인스턴스 생성
train_dataset = OverSamplingDataset(ng_dir='../PCB_imgs/all/resize/train/NG/', ok_dir='../PCB_imgs/all/resize/train/OK/', transform=transform, aug=aug)
validation_dataset = datasets.ImageFolder(root=val_dir, transform=transform)  # 검증, 테스트 데이터는 그대로 사용
test_dataset = datasets.ImageFolder(root=val_dir, transform=transform)

# DataLoader 설정
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

NG 이미지 수: 1178
OK 이미지 수: 830
오버샘플링 후 NG 이미지 수: 1178
오버샘플링 후 OK 이미지 수: 1178


In [33]:
# 모델 정의
class CustomModel(nn.Module):
    def __init__(self, model_name='efficientnet'):
        super(CustomModel, self).__init__()
        if model_name == 'efficientnet':
            self.base_model = torchvision.models.efficientnet_b1(weights='IMAGENET1K_V1')
            self.base_model.classifier = nn.Identity()  # 마지막 분류기 제거
        elif model_name == 'resnet50':
            self.base_model = torchvision.models.resnet50(weights='IMAGENET1K_V1')
            self.base_model = nn.Sequential(*list(self.base_model.children())[:-1])  # 마지막 레이어 제거
        elif model_name == 'inception':
            self.base_model = torchvision.models.inception_v3(weights='IMAGENET1K_V1')
            self.base_model.classifier = nn.Identity()  # 마지막 분류기 제거

        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(self._get_features_dim(model_name), 1)
        self.bn1 = nn.BatchNorm1d(self._get_features_dim(model_name))

    def _get_features_dim(self, model_name):
        if model_name == 'efficientnet':
            return 1280  # EfficientNet의 출력 차원
        elif model_name == 'resnet50':
            return 2048  # ResNet50의 출력 차원
        elif model_name == 'inception':
            return 2048  # Inception의 출력 차원

    def forward(self, x):
        x = self.base_model(x)
        if isinstance(x, tuple):  
            x = x[0] 
        
        x = x.view(x.size(0), -1) 
        x = self.bn1(x)
        x = self.dropout(x)
        x = self.fc1(x) 

        return x 

In [34]:
# 모델 초기화
model = CustomModel(model_name='efficientnet').to(DEVICE)
# 손실 함수 및 최적화함수 정의
criterion = nn.BCEWithLogitsLoss()  # 이진 교차 엔트로피 손실
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)  # Adam 옵티마이저

In [35]:
EPOCHS = 20

# 조기 종료 변수를 초기화합니다.
best_val_loss = float('inf')
patience = 5  # 개선이 없을 때 기다릴 에포크 수
patience_counter = 0

# 학습률 스케줄러
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5)

# 모델 훈련
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for images, targets in train_loader:
        images, targets = images.to(DEVICE), targets.to(DEVICE).float()
        
        optimizer.zero_grad()
        outputs = model(images)

        # 손실 계산
        loss = criterion(outputs.view(-1), targets)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

        # 정확도 계산
        predicted = (torch.sigmoid(outputs.view(-1)) > 0.5).float()
        correct_predictions += (predicted == targets).sum().item()
        total_predictions += targets.size(0)

    # 훈련 데이터 정확도
    train_accuracy = correct_predictions / total_predictions

    # 검증
    val_loss, val_accuracy = evaluate_model(model, validation_loader, criterion)

    # 학습률 스케줄러 적용
    scheduler.step(val_loss)

    print(f'Epoch [{epoch+1}/{EPOCHS}], Train Loss: {running_loss / len(train_loader):.4f}, '
          f'Train Accuracy: {train_accuracy:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

    # 조기 종료 로직
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0  # 손실 개선 시 카운터 리셋
        print("Validation loss improved, saving model...")
        # 모델 저장
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_loss': val_loss,
        }, '../model/efficientnet_b1_best_model.2.pth')
    else:
        patience_counter += 1  # 손실이 개선되지 않으면 카운터 증가

    if patience_counter >= patience:
        print("Early stopping triggered.")
        break  # 훈련 종료

Epoch [1/20], Train Loss: 0.4784, Train Accuracy: 0.7691, Val Loss: 0.3524, Val Accuracy: 0.8347
Validation loss improved, saving model...
Epoch [2/20], Train Loss: 0.3261, Train Accuracy: 0.8608, Val Loss: 0.2569, Val Accuracy: 0.8944
Validation loss improved, saving model...
Epoch [3/20], Train Loss: 0.2384, Train Accuracy: 0.8973, Val Loss: 0.2130, Val Accuracy: 0.9183
Validation loss improved, saving model...
Epoch [4/20], Train Loss: 0.1790, Train Accuracy: 0.9278, Val Loss: 0.1746, Val Accuracy: 0.9323
Validation loss improved, saving model...
Epoch [5/20], Train Loss: 0.1517, Train Accuracy: 0.9368, Val Loss: 0.1804, Val Accuracy: 0.9303
Epoch [6/20], Train Loss: 0.1345, Train Accuracy: 0.9491, Val Loss: 0.1464, Val Accuracy: 0.9442
Validation loss improved, saving model...
Epoch [7/20], Train Loss: 0.0999, Train Accuracy: 0.9622, Val Loss: 0.1531, Val Accuracy: 0.9323
Epoch [8/20], Train Loss: 0.0958, Train Accuracy: 0.9639, Val Loss: 0.1374, Val Accuracy: 0.9502
Validation los

In [40]:
# 모델 클래스 정의 (CustomModel)
model = CustomModel(model_name='efficientnet').to(DEVICE)

# 저장된 체크포인트 불러오기
checkpoint = torch.load('../model/efficientnet_b1_best_model.2.pth')

# 모델에 체크포인트 적용
model.load_state_dict(checkpoint['model_state_dict'])

optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)  # Adam 옵티마이저

# 모델을 평가 모드로 전환
model.eval()

C:\Users\enssel\AppData\Local\Temp\ipykernel_724\2733988964.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('../model/efficientnet_b1_best_model.

CustomModel(
  (base_model): EfficientNet(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): MBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
              (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
            (1): SqueezeExcitation(
              (avgpool): AdaptiveAvgPool2d(output_size=1)
              (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
              (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
              (activation): SiLU(inplace=True)
              (scal

In [41]:
# 테스트 데이터세트로 평가
test_loss, test_accuracy = evaluate_model(model, test_loader, criterion)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')

Test Loss: 0.1065, Test Accuracy: 0.9681


초기학습률 조정 (0.0001 -> 0.0005)

In [42]:
# 모델 초기화
model = CustomModel(model_name='efficientnet').to(DEVICE)
# 손실 함수 및 최적화함수 정의
criterion = nn.BCEWithLogitsLoss()  # 이진 교차 엔트로피 손실
optimizer = optim.Adam(model.parameters(), lr=0.0005, weight_decay=1e-4)  # Adam 옵티마이저

In [43]:
EPOCHS = 20

# 조기 종료 변수를 초기화합니다.
best_val_loss = float('inf')
patience = 5  # 개선이 없을 때 기다릴 에포크 수
patience_counter = 0

# 학습률 스케줄러
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5)

# 모델 훈련
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for images, targets in train_loader:
        images, targets = images.to(DEVICE), targets.to(DEVICE).float()
        
        optimizer.zero_grad()
        outputs = model(images)

        # 손실 계산
        loss = criterion(outputs.view(-1), targets)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

        # 정확도 계산
        predicted = (torch.sigmoid(outputs.view(-1)) > 0.5).float()
        correct_predictions += (predicted == targets).sum().item()
        total_predictions += targets.size(0)

    # 훈련 데이터 정확도
    train_accuracy = correct_predictions / total_predictions

    # 검증
    val_loss, val_accuracy = evaluate_model(model, validation_loader, criterion)

    # 학습률 스케줄러 적용
    scheduler.step(val_loss)

    print(f'Epoch [{epoch+1}/{EPOCHS}], Train Loss: {running_loss / len(train_loader):.4f}, '
          f'Train Accuracy: {train_accuracy:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

    # 조기 종료 로직
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0  # 손실 개선 시 카운터 리셋
        print("Validation loss improved, saving model...")
        # 모델 저장
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_loss': val_loss,
        }, '../model/efficientnet_b1_best_model.3.pth')
    else:
        patience_counter += 1  # 손실이 개선되지 않으면 카운터 증가

    if patience_counter >= patience:
        print("Early stopping triggered.")
        break  # 훈련 종료

Epoch [1/20], Train Loss: 0.3985, Train Accuracy: 0.8183, Val Loss: 0.2461, Val Accuracy: 0.9104
Validation loss improved, saving model...
Epoch [2/20], Train Loss: 0.2229, Train Accuracy: 0.9070, Val Loss: 0.1847, Val Accuracy: 0.9343
Validation loss improved, saving model...
Epoch [3/20], Train Loss: 0.1197, Train Accuracy: 0.9576, Val Loss: 0.1361, Val Accuracy: 0.9522
Validation loss improved, saving model...
Epoch [4/20], Train Loss: 0.1007, Train Accuracy: 0.9622, Val Loss: 0.1137, Val Accuracy: 0.9681
Validation loss improved, saving model...
Epoch [5/20], Train Loss: 0.0927, Train Accuracy: 0.9660, Val Loss: 0.1188, Val Accuracy: 0.9622
Epoch [6/20], Train Loss: 0.0559, Train Accuracy: 0.9796, Val Loss: 0.1127, Val Accuracy: 0.9701
Validation loss improved, saving model...
Epoch [7/20], Train Loss: 0.0435, Train Accuracy: 0.9868, Val Loss: 0.1299, Val Accuracy: 0.9602
Epoch [8/20], Train Loss: 0.0385, Train Accuracy: 0.9856, Val Loss: 0.1250, Val Accuracy: 0.9701
Epoch [9/20], 

In [44]:
# 모델 클래스 정의 (CustomModel)
model = CustomModel(model_name='efficientnet').to(DEVICE)

# 저장된 체크포인트 불러오기
checkpoint = torch.load('../model/efficientnet_b1_best_model.3.pth')

# 모델에 체크포인트 적용
model.load_state_dict(checkpoint['model_state_dict'])

optimizer = optim.Adam(model.parameters(), lr=0.0005, weight_decay=1e-4)  # Adam 옵티마이저

# 모델을 평가 모드로 전환
model.eval()

C:\Users\enssel\AppData\Local\Temp\ipykernel_724\2004558133.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('../model/efficientnet_b1_best_model.

CustomModel(
  (base_model): EfficientNet(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): MBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
              (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
            (1): SqueezeExcitation(
              (avgpool): AdaptiveAvgPool2d(output_size=1)
              (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
              (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
              (activation): SiLU(inplace=True)
              (scal

In [45]:
# 테스트 데이터세트로 평가
test_loss, test_accuracy = evaluate_model(model, test_loader, criterion)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')

Test Loss: 0.0757, Test Accuracy: 0.9801


##### Fine Tuning

In [46]:
from torchvision import datasets
import pandas as pd

IMAGE_SIZE = 240
BATCH_SIZE = 32
EPOCHS = 10

# 이미지 변환 정의
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),  # 이미지를 Tensor로 변환 (0~255 범위를 0~1 범위로 정규화)
])

# 데이터셋 디렉토리 설정
train_dir = '../PCB_imgs/all/resize/train'
val_dir = '../PCB_imgs/all/resize/validation'
test_dir = '../PCB_imgs/all/resize/test'

# ImageFolder로 데이터셋 불러오기
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
val_dataset = datasets.ImageFolder(root=val_dir, transform=transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)

# 파일 경로 및 타겟 추출
train_file_paths = [img[0] for img in train_dataset.imgs]
train_targets = train_dataset.targets

val_file_paths = [img[0] for img in val_dataset.imgs]
val_targets = val_dataset.targets

test_file_paths = [img[0] for img in test_dataset.imgs]
test_targets = test_dataset.targets

# DataFrame 생성
train_df = pd.DataFrame({'file_paths': train_file_paths, 'targets': train_targets})
validation_df = pd.DataFrame({'file_paths': val_file_paths, 'targets': val_targets})
test_df = pd.DataFrame({'file_paths': test_file_paths, 'targets': test_targets})

# 확인을 위해 각 데이터셋의 크기 출력
print(f"Train 데이터 수: {len(train_df)}")
print(f"Validation 데이터 수: {len(validation_df)}")
print(f"Test 데이터 수: {len(test_df)}")

Train 데이터 수: 2008
Validation 데이터 수: 502
Test 데이터 수: 628


In [63]:
import torch
from torchvision import datasets, transforms

# resize된 데이터셋 로드
input_dir = '../PCB_imgs/all/resize/all/'
dataset = datasets.ImageFolder(root=input_dir, transform=transform)

# 데이터 로더 정의
dataloader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

# 데이터 로더에서 데이터 가져오기
for images, labels in dataloader:
    print(images.shape)  # 배치의 이미지 텐서 크기 확인
    break  # 첫 번째 배치만 확인
    # 배치 크기, 채널 수(RGB), 이미지 크기 

torch.Size([32, 3, 240, 240])


In [64]:
IMAGE_SIZE = 240
BATCH_SIZE = 32
EPOCHS = 10

# 커스텀 데이터세트 정의
class CustomDataset(Dataset):
    def __init__(self, file_paths, targets, aug=None, preprocess=None):
        self.file_paths = file_paths
        self.targets = targets
        self.aug = aug
        self.preprocess = preprocess

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, index):
        file_path = self.file_paths[index]
        target = self.targets[index]
        
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        image = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE))

        if self.aug is not None:
            image = self.aug(image=image)['image']

        if self.preprocess is not None:
            image = self.preprocess(image)

        image = np.transpose(image, (2, 0, 1))  # (H, W, C) -> (C, H, W)
        image = torch.tensor(image, dtype=torch.float32)
        
        return image, target

In [65]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import DataLoader

# 데이터 증강 정의
aug = A.Compose([
    A.HorizontalFlip(p=0.5),                # 좌우 반전
    A.VerticalFlip(p=0.5),                  # 상하 반전
    A.Rotate(limit=10, p=0.5),              # 작은 각도 회전 (10도 내외)
    A.RandomBrightnessContrast(p=0.5),      # 밝기 및 대비 조절
])

# 데이터셋 인스턴스 생성
train_dataset = CustomDataset(train_df['file_paths'].values, train_df['targets'].values)
validation_dataset = CustomDataset(validation_df['file_paths'].values, validation_df['targets'].values)
test_dataset = CustomDataset(test_df['file_paths'].values, test_df['targets'].values)

# DataLoader 설정
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [66]:
# 모델 정의
class CustomModel(nn.Module):
    def __init__(self, model_name='efficientnet'):
        super(CustomModel, self).__init__()
        if model_name == 'efficientnet':
            self.base_model = torchvision.models.efficientnet_b1(weights='IMAGENET1K_V1')
            self.base_model.classifier = nn.Identity()  # 마지막 분류기 제거
        elif model_name == 'resnet50':
            self.base_model = torchvision.models.resnet50(weights='IMAGENET1K_V1')
            self.base_model = nn.Sequential(*list(self.base_model.children())[:-1])  # 마지막 레이어 제거
        elif model_name == 'inception':
            self.base_model = torchvision.models.inception_v3(weights='IMAGENET1K_V1')
            self.base_model.classifier = nn.Identity()  # 마지막 분류기 제거

        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(self._get_features_dim(model_name), 1)
        self.bn1 = nn.BatchNorm1d(self._get_features_dim(model_name))

    def _get_features_dim(self, model_name):
        if model_name == 'efficientnet':
            return 1280  # EfficientNet의 출력 차원
        elif model_name == 'resnet50':
            return 2048  # ResNet50의 출력 차원
        elif model_name == 'inception':
            return 2048  # Inception의 출력 차원

    def forward(self, x):
        x = self.base_model(x)
        if isinstance(x, tuple):  
            x = x[0] 
        
        x = x.view(x.size(0), -1) 
        x = self.bn1(x)
        x = self.dropout(x)
        x = self.fc1(x) 

        return x 

In [67]:
import torch.optim as optim

# 모델 생성
model = CustomModel(model_name='efficientnet').to(DEVICE)

# 사전 학습된 base_model의 파라미터를 동결 (fine-tuning 초기 단계)
for param in model.base_model.parameters():
    param.requires_grad = False

# 새로 추가된 분류기 레이어만 학습
optimizer = optim.Adam(model.fc1.parameters(), lr=1e-5) 

# 학습할 손실 함수
criterion = nn.BCEWithLogitsLoss()

# 각 층의 freeze/unfreeze 상태 확인
for name, param in model.named_parameters():
    print(f"Layer: {name} | Requires Grad: {param.requires_grad}")

Layer: base_model.features.0.0.weight | Requires Grad: False
Layer: base_model.features.0.1.weight | Requires Grad: False
Layer: base_model.features.0.1.bias | Requires Grad: False
Layer: base_model.features.1.0.block.0.0.weight | Requires Grad: False
Layer: base_model.features.1.0.block.0.1.weight | Requires Grad: False
Layer: base_model.features.1.0.block.0.1.bias | Requires Grad: False
Layer: base_model.features.1.0.block.1.fc1.weight | Requires Grad: False
Layer: base_model.features.1.0.block.1.fc1.bias | Requires Grad: False
Layer: base_model.features.1.0.block.1.fc2.weight | Requires Grad: False
Layer: base_model.features.1.0.block.1.fc2.bias | Requires Grad: False
Layer: base_model.features.1.0.block.2.0.weight | Requires Grad: False
Layer: base_model.features.1.0.block.2.1.weight | Requires Grad: False
Layer: base_model.features.1.0.block.2.1.bias | Requires Grad: False
Layer: base_model.features.1.1.block.0.0.weight | Requires Grad: False
Layer: base_model.features.1.1.block.0

In [68]:
EPOCHS = 50

# 조기 종료 변수 초기화
best_val_loss = float('inf')
patience = 5  # 개선이 없을 때 기다릴 에포크 수
patience_counter = 0

# 학습률 스케줄러
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5)

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for images, targets in train_loader:
        images, targets = images.to(DEVICE), targets.to(DEVICE).float()
        
        optimizer.zero_grad()
        outputs = model(images)

        # 손실 계산
        loss = criterion(outputs.view(-1), targets)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

        # 정확도 계산
        predicted = (torch.sigmoid(outputs.view(-1)) > 0.5).float()
        correct_predictions += (predicted == targets).sum().item()
        total_predictions += targets.size(0)

    # 훈련 데이터 정확도
    train_accuracy = correct_predictions / total_predictions

    # 검증
    val_loss, val_accuracy = evaluate_model(model, validation_loader, criterion)

    # 학습률 스케줄러 적용
    scheduler.step(val_loss)

    # 조기 종료 로직
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0  # 손실 개선 시 카운터 리셋
        print("Validation loss improved, saving model...")
        # 모델 저장
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_loss': val_loss,
        }, '../model/efficientnet_b1_ft_best_model_01.pth')
    else:
        patience_counter += 1  # 손실이 개선되지 않으면 카운터 증가

    if patience_counter >= patience:
        print("Early stopping triggered.")
        break  # 훈련 종료

    print(f'Epoch [{epoch+1}/{EPOCHS}], Train Loss: {running_loss / len(train_loader):.4f}, '
          f'Train Accuracy: {train_accuracy:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

Validation loss improved, saving model...
Epoch [1/50], Train Loss: 0.7507, Train Accuracy: 0.5199, Val Loss: 0.6949, Val Accuracy: 0.5299
Validation loss improved, saving model...
Epoch [2/50], Train Loss: 0.7193, Train Accuracy: 0.5528, Val Loss: 0.6871, Val Accuracy: 0.5478
Validation loss improved, saving model...
Epoch [3/50], Train Loss: 0.7058, Train Accuracy: 0.5583, Val Loss: 0.6666, Val Accuracy: 0.5976
Validation loss improved, saving model...
Epoch [4/50], Train Loss: 0.6942, Train Accuracy: 0.5872, Val Loss: 0.6539, Val Accuracy: 0.6215
Validation loss improved, saving model...
Epoch [5/50], Train Loss: 0.6742, Train Accuracy: 0.6026, Val Loss: 0.6383, Val Accuracy: 0.6434
Validation loss improved, saving model...
Epoch [6/50], Train Loss: 0.6674, Train Accuracy: 0.6096, Val Loss: 0.6231, Val Accuracy: 0.6514
Validation loss improved, saving model...
Epoch [7/50], Train Loss: 0.6504, Train Accuracy: 0.6389, Val Loss: 0.6180, Val Accuracy: 0.6793
Validation loss improved, s

In [71]:
# 모델 클래스 정의 (CustomModel)
model = CustomModel(model_name='efficientnet').to(DEVICE)

# 저장된 체크포인트 불러오기
checkpoint = torch.load('../model/efficientnet_b1_ft_best_model_01.pth')

# 모델에 체크포인트 적용
model.load_state_dict(checkpoint['model_state_dict'])

for param in model.base_model.parameters():
    param.requires_grad = False

# 새로 추가된 분류기 레이어만 학습
optimizer = optim.Adam(model.fc1.parameters(), lr=1e-5) 

# 에포크 정보 가져오기 (옵션)
start_epoch = checkpoint['epoch']

# 모델을 평가 모드로 전환
model.eval()

C:\Users\enssel\AppData\Local\Temp\ipykernel_724\684982325.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('../model/efficientnet_b1_ft_best_mode

CustomModel(
  (base_model): EfficientNet(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): MBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
              (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
            (1): SqueezeExcitation(
              (avgpool): AdaptiveAvgPool2d(output_size=1)
              (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
              (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
              (activation): SiLU(inplace=True)
              (scal

In [72]:
# 테스트 데이터세트로 평가
test_loss, test_accuracy = evaluate_model(model, test_loader, criterion)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')

Test Loss: 0.4527, Test Accuracy: 0.7755


##### 마지막 layer unfreeze

In [73]:
import torchvision.models as models

# 사전 학습된 모델 로드
model = models.efficientnet_b1(weights='IMAGENET1K_V1')

# 모델의 레이어 이름과 타입 확인
for name, layer in model.named_children():
    print(f"Layer Name: {name}, Layer Type: {layer}")

Layer Name: features, Layer Type: Sequential(
  (0): Conv2dNormActivation(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): SiLU(inplace=True)
  )
  (1): Sequential(
    (0): MBConv(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): SiLU(inplace=True)
        )
        (1): SqueezeExcitation(
          (avgpool): AdaptiveAvgPool2d(output_size=1)
          (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
          (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
          (activation): SiLU(inplace=True)
          (scale_activation): Sigmoid()
        )
        (2): Conv2dNormActivation(
          (0): Conv2d(32, 16, k

In [74]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import DataLoader

# 데이터 증강 정의
aug = A.Compose([
    A.HorizontalFlip(p=0.5),                # 좌우 반전
    A.VerticalFlip(p=0.5),                  # 상하 반전
    A.Rotate(limit=10, p=0.5),              # 작은 각도 회전 (10도 내외)
    A.RandomBrightnessContrast(p=0.5),      # 밝기 및 대비 조절
])

# 데이터셋 인스턴스 생성
train_dataset = CustomDataset(train_df['file_paths'].values, train_df['targets'].values, aug=aug)
validation_dataset = CustomDataset(validation_df['file_paths'].values, validation_df['targets'].values)
test_dataset = CustomDataset(test_df['file_paths'].values, test_df['targets'].values)

# DataLoader 설정
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [75]:
# 모든 레이어 이름 출력
for name, param in model.named_parameters():
    print(name)

features.0.0.weight
features.0.1.weight
features.0.1.bias
features.1.0.block.0.0.weight
features.1.0.block.0.1.weight
features.1.0.block.0.1.bias
features.1.0.block.1.fc1.weight
features.1.0.block.1.fc1.bias
features.1.0.block.1.fc2.weight
features.1.0.block.1.fc2.bias
features.1.0.block.2.0.weight
features.1.0.block.2.1.weight
features.1.0.block.2.1.bias
features.1.1.block.0.0.weight
features.1.1.block.0.1.weight
features.1.1.block.0.1.bias
features.1.1.block.1.fc1.weight
features.1.1.block.1.fc1.bias
features.1.1.block.1.fc2.weight
features.1.1.block.1.fc2.bias
features.1.1.block.2.0.weight
features.1.1.block.2.1.weight
features.1.1.block.2.1.bias
features.2.0.block.0.0.weight
features.2.0.block.0.1.weight
features.2.0.block.0.1.bias
features.2.0.block.1.0.weight
features.2.0.block.1.1.weight
features.2.0.block.1.1.bias
features.2.0.block.2.fc1.weight
features.2.0.block.2.fc1.bias
features.2.0.block.2.fc2.weight
features.2.0.block.2.fc2.bias
features.2.0.block.3.0.weight
features.2.0

In [76]:
# 모델 생성
model = CustomModel(model_name='efficientnet').to(DEVICE)

# 1. 사전 학습된 base_model의 파라미터를 동결 (fine-tuning 초기 단계)
for param in model.base_model.parameters():
    param.requires_grad = False

# 마지막 몇 개의 레이어를 동결 해제 (features 블록 8 이후 층)
for name, param in model.base_model.features[8:].named_parameters():
    param.requires_grad = True

# 동결 해제된 일부 층과 분류기 층 학습
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-5)

# 학습할 손실 함수
criterion = nn.BCEWithLogitsLoss()

# 각 층의 freeze/unfreeze 상태 확인
for name, param in model.named_parameters():
    print(f"Layer: {name} | Requires Grad: {param.requires_grad}")

Layer: base_model.features.0.0.weight | Requires Grad: False
Layer: base_model.features.0.1.weight | Requires Grad: False
Layer: base_model.features.0.1.bias | Requires Grad: False
Layer: base_model.features.1.0.block.0.0.weight | Requires Grad: False
Layer: base_model.features.1.0.block.0.1.weight | Requires Grad: False
Layer: base_model.features.1.0.block.0.1.bias | Requires Grad: False
Layer: base_model.features.1.0.block.1.fc1.weight | Requires Grad: False
Layer: base_model.features.1.0.block.1.fc1.bias | Requires Grad: False
Layer: base_model.features.1.0.block.1.fc2.weight | Requires Grad: False
Layer: base_model.features.1.0.block.1.fc2.bias | Requires Grad: False
Layer: base_model.features.1.0.block.2.0.weight | Requires Grad: False
Layer: base_model.features.1.0.block.2.1.weight | Requires Grad: False
Layer: base_model.features.1.0.block.2.1.bias | Requires Grad: False
Layer: base_model.features.1.1.block.0.0.weight | Requires Grad: False
Layer: base_model.features.1.1.block.0

In [77]:
EPOCHS = 50

# 조기 종료 변수 초기화
best_val_loss = float('inf')
patience = 5  # 개선이 없을 때 기다릴 에포크 수
patience_counter = 0

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for images, targets in train_loader:
        images, targets = images.to(DEVICE), targets.to(DEVICE).float()
        
        optimizer.zero_grad()
        outputs = model(images)

        # 손실 계산
        loss = criterion(outputs.view(-1), targets)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

        # 정확도 계산
        predicted = (torch.sigmoid(outputs.view(-1)) > 0.5).float()
        correct_predictions += (predicted == targets).sum().item()
        total_predictions += targets.size(0)

    # 훈련 데이터 정확도
    train_accuracy = correct_predictions / total_predictions

    # 검증
    val_loss, val_accuracy = evaluate_model(model, validation_loader, criterion)

    # 조기 종료 로직
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0  # 손실 개선 시 카운터 리셋
        print("Validation loss improved, saving model...")
        # 모델 저장
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_loss': val_loss,
        }, '../model/efficientnet_b1_ft_best_model_02.pth')
    else:
        patience_counter += 1  # 손실이 개선되지 않으면 카운터 증가

    if patience_counter >= patience:
        print("Early stopping triggered.")
        break  # 훈련 종료

    print(f'Epoch [{epoch+1}/{EPOCHS}], Train Loss: {running_loss / len(train_loader):.4f}, '
          f'Train Accuracy: {train_accuracy:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

Validation loss improved, saving model...
Epoch [1/50], Train Loss: 0.7608, Train Accuracy: 0.4970, Val Loss: 0.6991, Val Accuracy: 0.5159
Validation loss improved, saving model...
Epoch [2/50], Train Loss: 0.7167, Train Accuracy: 0.5513, Val Loss: 0.6582, Val Accuracy: 0.6036
Validation loss improved, saving model...
Epoch [3/50], Train Loss: 0.6661, Train Accuracy: 0.6071, Val Loss: 0.6198, Val Accuracy: 0.6793
Validation loss improved, saving model...
Epoch [4/50], Train Loss: 0.6416, Train Accuracy: 0.6355, Val Loss: 0.5931, Val Accuracy: 0.7112
Validation loss improved, saving model...
Epoch [5/50], Train Loss: 0.6201, Train Accuracy: 0.6509, Val Loss: 0.5707, Val Accuracy: 0.7131
Validation loss improved, saving model...
Epoch [6/50], Train Loss: 0.5817, Train Accuracy: 0.6833, Val Loss: 0.5576, Val Accuracy: 0.7291
Validation loss improved, saving model...
Epoch [7/50], Train Loss: 0.5756, Train Accuracy: 0.6858, Val Loss: 0.5339, Val Accuracy: 0.7450
Validation loss improved, s

In [92]:
model = CustomModel('efficientnet').to(DEVICE)

# 저장된 체크포인트 불러오기
checkpoint = torch.load('../model/efficientnet_b1_ft_best_model_02.pth')

# 모델에 체크포인트 적용
model.load_state_dict(checkpoint['model_state_dict'])

# 1. 사전 학습된 base_model의 파라미터를 동결 (fine-tuning 초기 단계)
for param in model.base_model.parameters():
    param.requires_grad = False

# 마지막 몇 개의 레이어를 동결 해제 (features 블록 8 이후 층)
for name, param in model.base_model.features[8:].named_parameters():
    param.requires_grad = True

# 동결 해제된 일부 층과 분류기 층 학습
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-5)

# 에포크 정보 가져오기 (옵션)
start_epoch = checkpoint['epoch']

# 모델을 평가 모드로 전환
model.eval()

C:\Users\enssel\AppData\Local\Temp\ipykernel_724\3408623219.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('../model/efficientnet_b1_ft_best_mod

CustomModel(
  (base_model): EfficientNet(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): MBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
              (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
            (1): SqueezeExcitation(
              (avgpool): AdaptiveAvgPool2d(output_size=1)
              (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
              (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
              (activation): SiLU(inplace=True)
              (scal

In [93]:
EPOCHS = 10

# 조기 종료 변수 초기화
best_val_loss = float('inf')
patience = 5  # 개선이 없을 때 기다릴 에포크 수
patience_counter = 0

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for images, targets in train_loader:
        images, targets = images.to(DEVICE), targets.to(DEVICE).float()
        
        optimizer.zero_grad()
        outputs = model(images)

        # 손실 계산
        loss = criterion(outputs.view(-1), targets)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

        # 정확도 계산
        predicted = (torch.sigmoid(outputs.view(-1)) > 0.5).float()
        correct_predictions += (predicted == targets).sum().item()
        total_predictions += targets.size(0)

    # 훈련 데이터 정확도
    train_accuracy = correct_predictions / total_predictions

    # 검증
    val_loss, val_accuracy = evaluate_model(model, validation_loader, criterion)

    # 조기 종료 로직
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0  # 손실 개선 시 카운터 리셋
        print("Validation loss improved, saving model...")
        # 모델 저장
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_loss': val_loss,
        }, '../model/efficientnet_b1_ft_best_model_02.pth')
    else:
        patience_counter += 1  # 손실이 개선되지 않으면 카운터 증가

    if patience_counter >= patience:
        print("Early stopping triggered.")
        break  # 훈련 종료

    print(f'Epoch [{epoch+1}/{EPOCHS}], Train Loss: {running_loss / len(train_loader):.4f}, '
          f'Train Accuracy: {train_accuracy:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

Validation loss improved, saving model...
Epoch [1/10], Train Loss: 0.1442, Train Accuracy: 0.9402, Val Loss: 0.1771, Val Accuracy: 0.9243
Epoch [2/10], Train Loss: 0.1321, Train Accuracy: 0.9432, Val Loss: 0.1887, Val Accuracy: 0.9223
Epoch [3/10], Train Loss: 0.1258, Train Accuracy: 0.9552, Val Loss: 0.1927, Val Accuracy: 0.9183
Epoch [4/10], Train Loss: 0.1282, Train Accuracy: 0.9452, Val Loss: 0.1800, Val Accuracy: 0.9183
Epoch [5/10], Train Loss: 0.1500, Train Accuracy: 0.9412, Val Loss: 0.1942, Val Accuracy: 0.9183
Early stopping triggered.


In [94]:
model = CustomModel('efficientnet').to(DEVICE)

# 저장된 체크포인트 불러오기
checkpoint = torch.load('../model/efficientnet_b1_ft_best_model_02.pth')

# 모델에 체크포인트 적용
model.load_state_dict(checkpoint['model_state_dict'])

# 1. 사전 학습된 base_model의 파라미터를 동결 (fine-tuning 초기 단계)
for param in model.base_model.parameters():
    param.requires_grad = False

# 마지막 몇 개의 레이어를 동결 해제 (features 블록 8 이후 층)
for name, param in model.base_model.features[8:].named_parameters():
    param.requires_grad = True

# 동결 해제된 일부 층과 분류기 층 학습
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-5)

# 에포크 정보 가져오기 (옵션)
start_epoch = checkpoint['epoch']

# 모델을 평가 모드로 전환
model.eval()

C:\Users\enssel\AppData\Local\Temp\ipykernel_724\3408623219.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('../model/efficientnet_b1_ft_best_mod

CustomModel(
  (base_model): EfficientNet(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): MBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
              (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
            (1): SqueezeExcitation(
              (avgpool): AdaptiveAvgPool2d(output_size=1)
              (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
              (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
              (activation): SiLU(inplace=True)
              (scal

In [95]:
# 테스트 데이터세트로 평가
test_loss, test_accuracy = evaluate_model(model, test_loader, criterion)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')

Test Loss: 0.1618, Test Accuracy: 0.9411


In [17]:
# 모델 생성
model = CustomModel(model_name='efficientnet').to(DEVICE)

# 1. 사전 학습된 base_model의 파라미터를 동결 (fine-tuning 초기 단계)
for param in model.base_model.parameters():
    param.requires_grad = False

# 마지막 몇 개의 레이어를 동결 해제 (features 블록 7 이후 층)
for name, param in model.base_model.features[7:].named_parameters():
    param.requires_grad = True

# 동결 해제된 일부 층과 분류기 층 학습
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-5)

# 학습할 손실 함수
criterion = nn.BCEWithLogitsLoss()

# 각 층의 freeze/unfreeze 상태 확인
for name, param in model.named_parameters():
    print(f"Layer: {name} | Requires Grad: {param.requires_grad}")

Layer: base_model.features.0.0.weight | Requires Grad: False
Layer: base_model.features.0.1.weight | Requires Grad: False
Layer: base_model.features.0.1.bias | Requires Grad: False
Layer: base_model.features.1.0.block.0.0.weight | Requires Grad: False
Layer: base_model.features.1.0.block.0.1.weight | Requires Grad: False
Layer: base_model.features.1.0.block.0.1.bias | Requires Grad: False
Layer: base_model.features.1.0.block.1.fc1.weight | Requires Grad: False
Layer: base_model.features.1.0.block.1.fc1.bias | Requires Grad: False
Layer: base_model.features.1.0.block.1.fc2.weight | Requires Grad: False
Layer: base_model.features.1.0.block.1.fc2.bias | Requires Grad: False
Layer: base_model.features.1.0.block.2.0.weight | Requires Grad: False
Layer: base_model.features.1.0.block.2.1.weight | Requires Grad: False
Layer: base_model.features.1.0.block.2.1.bias | Requires Grad: False
Layer: base_model.features.1.1.block.0.0.weight | Requires Grad: False
Layer: base_model.features.1.1.block.0

In [97]:
EPOCHS = 50

# 조기 종료 변수 초기화
best_val_loss = float('inf')
patience = 5  # 개선이 없을 때 기다릴 에포크 수
patience_counter = 0

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for images, targets in train_loader:
        images, targets = images.to(DEVICE), targets.to(DEVICE).float()
        
        optimizer.zero_grad()
        outputs = model(images)

        # 손실 계산
        loss = criterion(outputs.view(-1), targets)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

        # 정확도 계산
        predicted = (torch.sigmoid(outputs.view(-1)) > 0.5).float()
        correct_predictions += (predicted == targets).sum().item()
        total_predictions += targets.size(0)

    # 훈련 데이터 정확도
    train_accuracy = correct_predictions / total_predictions

    # 검증
    val_loss, val_accuracy = evaluate_model(model, validation_loader, criterion)

    # 조기 종료 로직
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0  # 손실 개선 시 카운터 리셋
        print("Validation loss improved, saving model...")
        # 모델 저장
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_loss': val_loss,
        }, '../model/efficientnet_b1_ft_best_model_03.pth')
    else:
        patience_counter += 1  # 손실이 개선되지 않으면 카운터 증가

    if patience_counter >= patience:
        print("Early stopping triggered.")
        break  # 훈련 종료

    print(f'Epoch [{epoch+1}/{EPOCHS}], Train Loss: {running_loss / len(train_loader):.4f}, '
          f'Train Accuracy: {train_accuracy:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

Validation loss improved, saving model...
Epoch [1/50], Train Loss: 0.7050, Train Accuracy: 0.5588, Val Loss: 0.6123, Val Accuracy: 0.6733
Validation loss improved, saving model...
Epoch [2/50], Train Loss: 0.6125, Train Accuracy: 0.6633, Val Loss: 0.5556, Val Accuracy: 0.7271
Validation loss improved, saving model...
Epoch [3/50], Train Loss: 0.5746, Train Accuracy: 0.6972, Val Loss: 0.5138, Val Accuracy: 0.7570
Validation loss improved, saving model...
Epoch [4/50], Train Loss: 0.5351, Train Accuracy: 0.7206, Val Loss: 0.4788, Val Accuracy: 0.7928
Validation loss improved, saving model...
Epoch [5/50], Train Loss: 0.5041, Train Accuracy: 0.7450, Val Loss: 0.4630, Val Accuracy: 0.8008
Validation loss improved, saving model...
Epoch [6/50], Train Loss: 0.4837, Train Accuracy: 0.7679, Val Loss: 0.4353, Val Accuracy: 0.8147
Validation loss improved, saving model...
Epoch [7/50], Train Loss: 0.4614, Train Accuracy: 0.7829, Val Loss: 0.4251, Val Accuracy: 0.8147
Validation loss improved, s

features 7 이후 블록을 unfreeze 한 후, 학습하였으나 이전보다 성능이 더 낮아짐.  ???????   
features 8 이후 블록을 unfreeze 하되, 데이터 증강, ReLU함수 추가

In [3]:
# 모델 정의
class CustomModel(nn.Module):
    def __init__(self, model_name='efficientnet'):
        super(CustomModel, self).__init__()
        if model_name == 'efficientnet':
            self.base_model = torchvision.models.efficientnet_b1(weights='IMAGENET1K_V1')
            self.base_model.classifier = nn.Identity()  # 마지막 분류기 제거
        elif model_name == 'resnet50':
            self.base_model = torchvision.models.resnet50(weights='IMAGENET1K_V1')
            self.base_model = nn.Sequential(*list(self.base_model.children())[:-1])  # 마지막 레이어 제거
        elif model_name == 'inception':
            self.base_model = torchvision.models.inception_v3(weights='IMAGENET1K_V1')
            self.base_model.classifier = nn.Identity()  # 마지막 분류기 제거

        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(self._get_features_dim(model_name), 50)
        self.bn1 = nn.BatchNorm1d(50)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(50, 1)

    def _get_features_dim(self, model_name):
        if model_name == 'efficientnet':
            return 1280  # EfficientNet B0의 출력 차원
        elif model_name == 'resnet50':
            return 2048  # ResNet50의 출력 차원
        elif model_name == 'inception':
            return 2048  # Inception의 출력 차원

    def forward(self, x):
        x = self.base_model(x)
        
        # Inception 모델에 대한 수정
        if isinstance(x, tuple):  # Inception 모델이 여러 출력을 반환하는 경우
            x = x[0]  # 첫 번째 출력을 선택
        
        x = x.view(x.size(0), -1)  # Flatten
        x = self.dropout(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.bn1(x)
        x = self.dropout(x)
        x = self.fc2(x)

        return x  # 최종 출력

In [25]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import DataLoader

# 데이터 증강 정의
aug = A.Compose([
    A.HorizontalFlip(p=0.5),                # 좌우 반전
    A.VerticalFlip(p=0.5),                  # 상하 반전
    A.Rotate(limit=10, p=0.5),              # 작은 각도 회전 (10도 내외)
    A.RandomBrightnessContrast(p=0.5),      # 밝기 및 대비 조절
])

# 데이터셋 인스턴스 생성
train_dataset = CustomDataset(train_df['file_paths'].values, train_df['targets'].values, aug=aug)
validation_dataset = CustomDataset(validation_df['file_paths'].values, validation_df['targets'].values)
test_dataset = CustomDataset(test_df['file_paths'].values, test_df['targets'].values)

# DataLoader 설정
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [26]:
# 모델 생성
model = CustomModel(model_name='efficientnet').to(DEVICE)

# 1. 사전 학습된 base_model의 파라미터를 동결 (fine-tuning 초기 단계)
for param in model.base_model.parameters():
    param.requires_grad = False

# 마지막 몇 개의 레이어를 동결 해제 (features 블록 8 이후 층)
for name, param in model.base_model.features[8:].named_parameters():
    param.requires_grad = True

# 동결 해제된 일부 층과 분류기 층 학습
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-5)

# 학습할 손실 함수
criterion = nn.BCEWithLogitsLoss()

# 각 층의 freeze/unfreeze 상태 확인
for name, param in model.named_parameters():
    print(f"Layer: {name} | Requires Grad: {param.requires_grad}")

Layer: base_model.features.0.0.weight | Requires Grad: False
Layer: base_model.features.0.1.weight | Requires Grad: False
Layer: base_model.features.0.1.bias | Requires Grad: False
Layer: base_model.features.1.0.block.0.0.weight | Requires Grad: False
Layer: base_model.features.1.0.block.0.1.weight | Requires Grad: False
Layer: base_model.features.1.0.block.0.1.bias | Requires Grad: False
Layer: base_model.features.1.0.block.1.fc1.weight | Requires Grad: False
Layer: base_model.features.1.0.block.1.fc1.bias | Requires Grad: False
Layer: base_model.features.1.0.block.1.fc2.weight | Requires Grad: False
Layer: base_model.features.1.0.block.1.fc2.bias | Requires Grad: False
Layer: base_model.features.1.0.block.2.0.weight | Requires Grad: False
Layer: base_model.features.1.0.block.2.1.weight | Requires Grad: False
Layer: base_model.features.1.0.block.2.1.bias | Requires Grad: False
Layer: base_model.features.1.1.block.0.0.weight | Requires Grad: False
Layer: base_model.features.1.1.block.0

In [27]:
EPOCHS = 50

# 조기 종료 변수 초기화
best_val_loss = float('inf')
patience = 5  # 개선이 없을 때 기다릴 에포크 수
patience_counter = 0

# 학습률 스케줄러
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5)

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for images, targets in train_loader:
        images, targets = images.to(DEVICE), targets.to(DEVICE).float()
        
        optimizer.zero_grad()
        outputs = model(images)

        # 손실 계산
        loss = criterion(outputs.view(-1), targets)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

        # 정확도 계산
        predicted = (torch.sigmoid(outputs.view(-1)) > 0.5).float()
        correct_predictions += (predicted == targets).sum().item()
        total_predictions += targets.size(0)

    # 훈련 데이터 정확도
    train_accuracy = correct_predictions / total_predictions

    # 검증
    val_loss, val_accuracy = evaluate_model(model, validation_loader, criterion)

    # 학습률 스케줄러 적용
    scheduler.step(val_loss)

    # 현재 학습률 출력
    current_lr = optimizer.param_groups[0]['lr']

    # 조기 종료 로직
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0  # 손실 개선 시 카운터 리셋
        print("Validation loss improved, saving model...")
        # 모델 저장
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_loss': val_loss,
        }, '../model/efficientnet_b1_ft_best_model_04.pth')
    else:
        patience_counter += 1  # 손실이 개선되지 않으면 카운터 증가

    if patience_counter >= patience:
        print("Early stopping triggered.")
        break  # 훈련 종료

    # 에포크별 정보 출력 (학습률 포함)
    print(f'Epoch [{epoch+1}/{EPOCHS}], Train Loss: {running_loss / len(train_loader):.4f}, '
          f'Train Accuracy: {train_accuracy:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}, '
          f'Learning Rate: {current_lr:.6f}')

Validation loss improved, saving model...
Epoch [1/50], Train Loss: 0.7569, Train Accuracy: 0.5085, Val Loss: 0.6839, Val Accuracy: 0.5319, Learning Rate: 0.000010
Validation loss improved, saving model...
Epoch [2/50], Train Loss: 0.7171, Train Accuracy: 0.5518, Val Loss: 0.6505, Val Accuracy: 0.6335, Learning Rate: 0.000010
Validation loss improved, saving model...
Epoch [3/50], Train Loss: 0.6722, Train Accuracy: 0.6061, Val Loss: 0.6169, Val Accuracy: 0.7131, Learning Rate: 0.000010
Validation loss improved, saving model...
Epoch [4/50], Train Loss: 0.6529, Train Accuracy: 0.6150, Val Loss: 0.5940, Val Accuracy: 0.7390, Learning Rate: 0.000010
Validation loss improved, saving model...
Epoch [5/50], Train Loss: 0.6302, Train Accuracy: 0.6340, Val Loss: 0.5703, Val Accuracy: 0.7550, Learning Rate: 0.000010
Validation loss improved, saving model...
Epoch [6/50], Train Loss: 0.6132, Train Accuracy: 0.6584, Val Loss: 0.5538, Val Accuracy: 0.7649, Learning Rate: 0.000010
Validation loss 

In [28]:
model = CustomModel('efficientnet').to(DEVICE)

checkpoint = torch.load('../model/efficientnet_b1_ft_best_model_04.pth')

model.load_state_dict(checkpoint['model_state_dict'])

for param in model.base_model.parameters():
    param.requires_grad = False

# 마지막 몇 개의 레이어를 동결 해제 (features 블록 8 이후 층)
for name, param in model.base_model.features[8:].named_parameters():
    param.requires_grad = True

# 동결 해제된 일부 층과 분류기 층 학습
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-5)

model.eval()

C:\Users\enssel\AppData\Local\Temp\ipykernel_16704\3511494907.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('../model/efficientnet_b1_ft_best_m

CustomModel(
  (base_model): EfficientNet(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): MBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
              (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
            (1): SqueezeExcitation(
              (avgpool): AdaptiveAvgPool2d(output_size=1)
              (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
              (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
              (activation): SiLU(inplace=True)
              (scal

In [29]:
test_loss, test_accuracy = evaluate_model(model, test_loader, criterion)
print(f'Test loss: {test_loss:.4f}, Test accuracy: {test_accuracy:.4f}')

Test loss: 0.3839, Test accuracy: 0.8185


추가 50 epoch

In [30]:
EPOCHS = 50

# 조기 종료 변수 초기화
best_val_loss = float('inf')
patience = 5  # 개선이 없을 때 기다릴 에포크 수
patience_counter = 0

# 학습률 스케줄러
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5)

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for images, targets in train_loader:
        images, targets = images.to(DEVICE), targets.to(DEVICE).float()
        
        optimizer.zero_grad()
        outputs = model(images)

        # 손실 계산
        loss = criterion(outputs.view(-1), targets)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

        # 정확도 계산
        predicted = (torch.sigmoid(outputs.view(-1)) > 0.5).float()
        correct_predictions += (predicted == targets).sum().item()
        total_predictions += targets.size(0)

    # 훈련 데이터 정확도
    train_accuracy = correct_predictions / total_predictions

    # 검증
    val_loss, val_accuracy = evaluate_model(model, validation_loader, criterion)

    # 학습률 스케줄러 적용
    scheduler.step(val_loss)

    # 현재 학습률 출력
    current_lr = optimizer.param_groups[0]['lr']

    # 조기 종료 로직
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0  # 손실 개선 시 카운터 리셋
        print("Validation loss improved, saving model...")
        # 모델 저장
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_loss': val_loss,
        }, '../model/efficientnet_b1_ft_best_model_04.pth')
    else:
        patience_counter += 1  # 손실이 개선되지 않으면 카운터 증가

    if patience_counter >= patience:
        print("Early stopping triggered.")
        break  # 훈련 종료

    # 에포크별 정보 출력 (학습률 포함)
    print(f'Epoch [{epoch+1}/{EPOCHS}], Train Loss: {running_loss / len(train_loader):.4f}, '
          f'Train Accuracy: {train_accuracy:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}, '
          f'Learning Rate: {current_lr:.6f}')

Validation loss improved, saving model...
Epoch [1/50], Train Loss: 0.4260, Train Accuracy: 0.7923, Val Loss: 0.3840, Val Accuracy: 0.8386, Learning Rate: 0.000010
Epoch [2/50], Train Loss: 0.4291, Train Accuracy: 0.7978, Val Loss: 0.3878, Val Accuracy: 0.8406, Learning Rate: 0.000010
Validation loss improved, saving model...
Epoch [3/50], Train Loss: 0.4102, Train Accuracy: 0.8068, Val Loss: 0.3739, Val Accuracy: 0.8406, Learning Rate: 0.000010
Epoch [4/50], Train Loss: 0.4195, Train Accuracy: 0.8063, Val Loss: 0.3785, Val Accuracy: 0.8446, Learning Rate: 0.000010
Epoch [5/50], Train Loss: 0.4190, Train Accuracy: 0.7958, Val Loss: 0.3788, Val Accuracy: 0.8446, Learning Rate: 0.000010
Epoch [6/50], Train Loss: 0.4080, Train Accuracy: 0.8038, Val Loss: 0.3780, Val Accuracy: 0.8406, Learning Rate: 0.000010
Epoch [7/50], Train Loss: 0.4167, Train Accuracy: 0.8053, Val Loss: 0.3772, Val Accuracy: 0.8466, Learning Rate: 0.000005
Validation loss improved, saving model...
Epoch [8/50], Train 

In [31]:
model = CustomModel('efficientnet').to(DEVICE)

checkpoint = torch.load('../model/efficientnet_b1_ft_best_model_04.pth')

model.load_state_dict(checkpoint['model_state_dict'])

for param in model.base_model.parameters():
    param.requires_grad = False

# 마지막 몇 개의 레이어를 동결 해제 (features 블록 8 이후 층)
for name, param in model.base_model.features[8:].named_parameters():
    param.requires_grad = True

# 동결 해제된 일부 층과 분류기 층 학습
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-5)

model.eval()

C:\Users\enssel\AppData\Local\Temp\ipykernel_16704\3511494907.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('../model/efficientnet_b1_ft_best_m

CustomModel(
  (base_model): EfficientNet(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): MBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
              (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
            (1): SqueezeExcitation(
              (avgpool): AdaptiveAvgPool2d(output_size=1)
              (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
              (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
              (activation): SiLU(inplace=True)
              (scal

In [32]:
test_loss, test_accuracy = evaluate_model(model, test_loader, criterion)
print(f'Test loss: {test_loss:.4f}, Test accuracy: {test_accuracy:.4f}')

Test loss: 0.3640, Test accuracy: 0.8392


여전히 가벼운 모델..?  
feature 5 ~ unfreeze

In [33]:
class CustomModel(nn.Module):
    def __init__(self, model_name='efficientnet'):
        super(CustomModel, self).__init__()
        if model_name == 'efficientnet':
            self.base_model = torchvision.models.efficientnet_b1(weights='IMAGENET1K_V1')
            self.base_model.classifier = nn.Identity()  # 마지막 분류기 제거
        elif model_name == 'resnet50':
            self.base_model = torchvision.models.resnet50(weights='IMAGENET1K_V1')
            self.base_model = nn.Sequential(*list(self.base_model.children())[:-1])  # 마지막 레이어 제거
        elif model_name == 'inception':
            self.base_model = torchvision.models.inception_v3(weights='IMAGENET1K_V1')
            self.base_model.classifier = nn.Identity()  # 마지막 분류기 제거

        self.dropout1 = nn.Dropout(0.5)  # 첫 번째 드롭아웃
        self.fc1 = nn.Linear(self._get_features_dim(model_name), 50)
        self.bn1 = nn.BatchNorm1d(50)
        self.relu = nn.ReLU()
        self.dropout2 = nn.Dropout(0.5)  # 두 번째 드롭아웃
        self.fc2 = nn.Linear(50, 1)

    def _get_features_dim(self, model_name):
        if model_name == 'efficientnet':
            return 1280  # EfficientNet B0의 출력 차원
        elif model_name == 'resnet50':
            return 2048  # ResNet50의 출력 차원
        elif model_name == 'inception':
            return 2048  # Inception의 출력 차원

    def forward(self, x):
        x = self.base_model(x)

        if isinstance(x, tuple):  
            x = x[0]
        
        x = x.view(x.size(0), -1)  # Flatten
        
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout1(x) 
        x = self.bn1(x)  # 배치 정규화 적용
        x = self.dropout2(x) 
        x = self.fc2(x)
        
        return x 

In [34]:
# 모델 생성
model = CustomModel(model_name='efficientnet').to(DEVICE)

# 1. 사전 학습된 base_model의 파라미터를 동결 (fine-tuning 초기 단계)
for param in model.base_model.parameters():
    param.requires_grad = False

# 마지막 몇 개의 레이어를 동결 해제 (features 블록 5 이후 층)
for name, param in model.base_model.features[5:].named_parameters():
    param.requires_grad = True

# 동결 해제된 일부 층과 분류기 층 학습
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=5e-5)

# 학습할 손실 함수
criterion = nn.BCEWithLogitsLoss()

# 각 층의 freeze/unfreeze 상태 확인
for name, param in model.named_parameters():
    print(f"Layer: {name} | Requires Grad: {param.requires_grad}")

Layer: base_model.features.0.0.weight | Requires Grad: False
Layer: base_model.features.0.1.weight | Requires Grad: False
Layer: base_model.features.0.1.bias | Requires Grad: False
Layer: base_model.features.1.0.block.0.0.weight | Requires Grad: False
Layer: base_model.features.1.0.block.0.1.weight | Requires Grad: False
Layer: base_model.features.1.0.block.0.1.bias | Requires Grad: False
Layer: base_model.features.1.0.block.1.fc1.weight | Requires Grad: False
Layer: base_model.features.1.0.block.1.fc1.bias | Requires Grad: False
Layer: base_model.features.1.0.block.1.fc2.weight | Requires Grad: False
Layer: base_model.features.1.0.block.1.fc2.bias | Requires Grad: False
Layer: base_model.features.1.0.block.2.0.weight | Requires Grad: False
Layer: base_model.features.1.0.block.2.1.weight | Requires Grad: False
Layer: base_model.features.1.0.block.2.1.bias | Requires Grad: False
Layer: base_model.features.1.1.block.0.0.weight | Requires Grad: False
Layer: base_model.features.1.1.block.0

In [35]:
EPOCHS = 50

# 조기 종료 변수 초기화
best_val_loss = float('inf')
patience = 5  # 개선이 없을 때 기다릴 에포크 수
patience_counter = 0

# 학습률 스케줄러
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5)

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for images, targets in train_loader:
        images, targets = images.to(DEVICE), targets.to(DEVICE).float()
        
        optimizer.zero_grad()
        outputs = model(images)

        # 손실 계산
        loss = criterion(outputs.view(-1), targets)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

        # 정확도 계산
        predicted = (torch.sigmoid(outputs.view(-1)) > 0.5).float()
        correct_predictions += (predicted == targets).sum().item()
        total_predictions += targets.size(0)

    # 훈련 데이터 정확도
    train_accuracy = correct_predictions / total_predictions

    # 검증
    val_loss, val_accuracy = evaluate_model(model, validation_loader, criterion)

    # 학습률 스케줄러 적용
    scheduler.step(val_loss)

    # 현재 학습률 출력
    current_lr = optimizer.param_groups[0]['lr']

    # 조기 종료 로직
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0  # 손실 개선 시 카운터 리셋
        print("Validation loss improved, saving model...")
        # 모델 저장
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_loss': val_loss,
        }, '../model/efficientnet_b1_ft_best_model_05.1.pth')
    else:
        patience_counter += 1  # 손실이 개선되지 않으면 카운터 증가

    if patience_counter >= patience:
        print("Early stopping triggered.")
        break  # 훈련 종료

    # 에포크별 정보 출력 (학습률 포함)
    print(f'Epoch [{epoch+1}/{EPOCHS}], Train Loss: {running_loss / len(train_loader):.4f}, '
          f'Train Accuracy: {train_accuracy:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}, '
          f'Learning Rate: {current_lr:.6f}')

Validation loss improved, saving model...
Epoch [1/50], Train Loss: 0.6282, Train Accuracy: 0.6474, Val Loss: 0.4945, Val Accuracy: 0.8048, Learning Rate: 0.000050
Validation loss improved, saving model...
Epoch [2/50], Train Loss: 0.5286, Train Accuracy: 0.7356, Val Loss: 0.4360, Val Accuracy: 0.8347, Learning Rate: 0.000050
Validation loss improved, saving model...
Epoch [3/50], Train Loss: 0.4771, Train Accuracy: 0.7764, Val Loss: 0.3998, Val Accuracy: 0.8566, Learning Rate: 0.000050
Validation loss improved, saving model...
Epoch [4/50], Train Loss: 0.4431, Train Accuracy: 0.7988, Val Loss: 0.3793, Val Accuracy: 0.8645, Learning Rate: 0.000050
Validation loss improved, saving model...
Epoch [5/50], Train Loss: 0.4210, Train Accuracy: 0.8123, Val Loss: 0.3610, Val Accuracy: 0.8884, Learning Rate: 0.000050
Validation loss improved, saving model...
Epoch [6/50], Train Loss: 0.4006, Train Accuracy: 0.8272, Val Loss: 0.3551, Val Accuracy: 0.8805, Learning Rate: 0.000050
Validation loss 

In [36]:
model = CustomModel('efficientnet').to(DEVICE)

checkpoint = torch.load('../model/efficientnet_b1_ft_best_model_05.1.pth')

model.load_state_dict(checkpoint['model_state_dict'])

for param in model.base_model.parameters():
    param.requires_grad = False

# 마지막 몇 개의 레이어를 동결 해제 (features 블록 5 이후 층)
for name, param in model.base_model.features[5:].named_parameters():
    param.requires_grad = True

# 동결 해제된 일부 층과 분류기 층 학습
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=5e-5)

model.eval()

C:\Users\enssel\AppData\Local\Temp\ipykernel_16704\2454304535.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('../model/efficientnet_b1_ft_best_m

CustomModel(
  (base_model): EfficientNet(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): MBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
              (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
            (1): SqueezeExcitation(
              (avgpool): AdaptiveAvgPool2d(output_size=1)
              (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
              (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
              (activation): SiLU(inplace=True)
              (scal

In [37]:
test_loss, test_accuracy = evaluate_model(model, test_loader, criterion)
print(f'Test loss: {test_loss:.4f}, Test accuracy: {test_accuracy:.4f}')

Test loss: 0.1147, Test accuracy: 0.9618


!!!!!

efficientNet b0랑 똑같이

In [8]:
class CustomModel(nn.Module):
    def __init__(self, model_name='efficientnet'):
        super(CustomModel, self).__init__()
        if model_name == 'efficientnet':
            self.base_model = torchvision.models.efficientnet_b1(weights='IMAGENET1K_V1')
            self.base_model.classifier = nn.Identity()  # 마지막 분류기 제거
        elif model_name == 'resnet50':
            self.base_model = torchvision.models.resnet50(weights='IMAGENET1K_V1')
            self.base_model = nn.Sequential(*list(self.base_model.children())[:-1])  # 마지막 레이어 제거
        elif model_name == 'inception':
            self.base_model = torchvision.models.inception_v3(weights='IMAGENET1K_V1')
            self.base_model.classifier = nn.Identity()  # 마지막 분류기 제거

        self.dropout1 = nn.Dropout(0.5)  # 첫 번째 드롭아웃
        self.fc1 = nn.Linear(self._get_features_dim(model_name), 50)
        self.bn1 = nn.BatchNorm1d(50)
        self.relu = nn.ReLU()
        self.dropout2 = nn.Dropout(0.5)  # 두 번째 드롭아웃
        self.fc2 = nn.Linear(50, 1)

    def _get_features_dim(self, model_name):
        if model_name == 'efficientnet':
            return 1280  # EfficientNet B0의 출력 차원
        elif model_name == 'resnet50':
            return 2048  # ResNet50의 출력 차원
        elif model_name == 'inception':
            return 2048  # Inception의 출력 차원

    def forward(self, x):
        x = self.base_model(x)

        if isinstance(x, tuple):  
            x = x[0]
        
        x = x.view(x.size(0), -1)  # Flatten
        
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout1(x) 
        x = self.bn1(x)  # 배치 정규화 적용
        x = self.dropout2(x) 
        x = self.fc2(x)
        
        return x 

In [9]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import DataLoader

# 데이터 증강 정의
aug = A.Compose([
    A.HorizontalFlip(p=0.5),                # 좌우 반전
    A.VerticalFlip(p=0.5),                  # 상하 반전
    A.Rotate(limit=10, p=0.5),              # 작은 각도 회전 (10도 내외)
    A.RandomBrightnessContrast(p=0.5),      # 밝기 및 대비 조절
])

# 데이터셋 인스턴스 생성
train_dataset = CustomDataset(train_df['file_paths'].values, train_df['targets'].values, aug=aug)
validation_dataset = CustomDataset(validation_df['file_paths'].values, validation_df['targets'].values)
test_dataset = CustomDataset(test_df['file_paths'].values, test_df['targets'].values)

# DataLoader 설정
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [10]:
# 모델 생성
model = CustomModel(model_name='efficientnet').to(DEVICE)

# 1. 사전 학습된 base_model의 파라미터를 동결 (fine-tuning 초기 단계)
for param in model.base_model.parameters():
    param.requires_grad = False

# 마지막 몇 개의 레이어를 동결 해제 (features 블록 5 이후 층)
for name, param in model.base_model.features[5:].named_parameters():
    param.requires_grad = True

# 동결 해제된 일부 층과 분류기 층 학습
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=5e-5)

# 학습할 손실 함수
criterion = nn.BCEWithLogitsLoss()

# 각 층의 freeze/unfreeze 상태 확인
for name, param in model.named_parameters():
    print(f"Layer: {name} | Requires Grad: {param.requires_grad}")

Layer: base_model.features.0.0.weight | Requires Grad: False
Layer: base_model.features.0.1.weight | Requires Grad: False
Layer: base_model.features.0.1.bias | Requires Grad: False
Layer: base_model.features.1.0.block.0.0.weight | Requires Grad: False
Layer: base_model.features.1.0.block.0.1.weight | Requires Grad: False
Layer: base_model.features.1.0.block.0.1.bias | Requires Grad: False
Layer: base_model.features.1.0.block.1.fc1.weight | Requires Grad: False
Layer: base_model.features.1.0.block.1.fc1.bias | Requires Grad: False
Layer: base_model.features.1.0.block.1.fc2.weight | Requires Grad: False
Layer: base_model.features.1.0.block.1.fc2.bias | Requires Grad: False
Layer: base_model.features.1.0.block.2.0.weight | Requires Grad: False
Layer: base_model.features.1.0.block.2.1.weight | Requires Grad: False
Layer: base_model.features.1.0.block.2.1.bias | Requires Grad: False
Layer: base_model.features.1.1.block.0.0.weight | Requires Grad: False
Layer: base_model.features.1.1.block.0

In [11]:
EPOCHS = 50

# 조기 종료 변수 초기화
best_val_loss = float('inf')
patience = 5  # 개선이 없을 때 기다릴 에포크 수
patience_counter = 0

# 학습률 스케줄러
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5)

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for images, targets in train_loader:
        images, targets = images.to(DEVICE), targets.to(DEVICE).float()
        
        optimizer.zero_grad()
        outputs = model(images)

        # 손실 계산
        loss = criterion(outputs.view(-1), targets)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

        # 정확도 계산
        predicted = (torch.sigmoid(outputs.view(-1)) > 0.5).float()
        correct_predictions += (predicted == targets).sum().item()
        total_predictions += targets.size(0)

    # 훈련 데이터 정확도
    train_accuracy = correct_predictions / total_predictions

    # 검증
    val_loss, val_accuracy = evaluate_model(model, validation_loader, criterion)

    # 학습률 스케줄러 적용
    scheduler.step(val_loss)

    # 현재 학습률 출력
    current_lr = optimizer.param_groups[0]['lr']

    # 조기 종료 로직
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0  # 손실 개선 시 카운터 리셋
        print("Validation loss improved, saving model...")
        # 모델 저장
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_loss': val_loss,
        }, '../model/efficientnet_b1_ft_best_model_06.pth')
    else:
        patience_counter += 1  # 손실이 개선되지 않으면 카운터 증가

    if patience_counter >= patience:
        print("Early stopping triggered.")
        break  # 훈련 종료

    # 에포크별 정보 출력 (학습률 포함)
    print(f'Epoch [{epoch+1}/{EPOCHS}], Train Loss: {running_loss / len(train_loader):.4f}, '
          f'Train Accuracy: {train_accuracy:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}, '
          f'Learning Rate: {current_lr:.6f}')

Validation loss improved, saving model...
Epoch [1/50], Train Loss: 0.6720, Train Accuracy: 0.5881, Val Loss: 0.5291, Val Accuracy: 0.7809, Learning Rate: 0.000050
Validation loss improved, saving model...
Epoch [2/50], Train Loss: 0.5463, Train Accuracy: 0.7186, Val Loss: 0.4588, Val Accuracy: 0.8147, Learning Rate: 0.000050
Validation loss improved, saving model...
Epoch [3/50], Train Loss: 0.4865, Train Accuracy: 0.7629, Val Loss: 0.4293, Val Accuracy: 0.8347, Learning Rate: 0.000050
Validation loss improved, saving model...
Epoch [4/50], Train Loss: 0.4675, Train Accuracy: 0.7799, Val Loss: 0.4020, Val Accuracy: 0.8367, Learning Rate: 0.000050
Validation loss improved, saving model...
Epoch [5/50], Train Loss: 0.4289, Train Accuracy: 0.8093, Val Loss: 0.3772, Val Accuracy: 0.8546, Learning Rate: 0.000050
Validation loss improved, saving model...
Epoch [6/50], Train Loss: 0.4148, Train Accuracy: 0.8252, Val Loss: 0.3625, Val Accuracy: 0.8606, Learning Rate: 0.000050
Validation loss 

In [12]:
model = CustomModel('efficientnet').to(DEVICE)

checkpoint = torch.load('../model/efficientnet_b1_ft_best_model_06.pth')

model.load_state_dict(checkpoint['model_state_dict'])

for param in model.base_model.parameters():
    param.requires_grad = False

# 마지막 몇 개의 레이어를 동결 해제 (features 블록 5 이후 층)
for name, param in model.base_model.features[5:].named_parameters():
    param.requires_grad = True

# 동결 해제된 일부 층과 분류기 층 학습
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=5e-5)

model.eval()

C:\Users\enssel\AppData\Local\Temp\ipykernel_1892\3854422640.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('../model/efficientnet_b1_ft_best_mo

CustomModel(
  (base_model): EfficientNet(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): MBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
              (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
            (1): SqueezeExcitation(
              (avgpool): AdaptiveAvgPool2d(output_size=1)
              (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
              (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
              (activation): SiLU(inplace=True)
              (scal

In [13]:
test_loss, test_accuracy = evaluate_model(model, test_loader, criterion)
print(f'Test loss: {test_loss:.4f}, Test accuracy: {test_accuracy:.4f}')

Test loss: 0.1270, Test accuracy: 0.9586
